<a href="https://colab.research.google.com/github/UrukHan/NeuroNetStockQuotes/blob/master/DownloadApiData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Импорт библиотек**

In [19]:
!pip install alpha_vantage

In [20]:
import requests
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from alpha_vantage.timeseries import TimeSeries
from datetime import datetime

In [21]:
# Clear logs from previous calls
!rm -rf ./logs/ 

# Check
!ls

CompaniesList.txt  sample_data


### **Скачивание данных через API с https://www.alphavantage.co/**

In [22]:
# Ввод ключа (IQLCQLXYBPW3LH7T)
key = 'IQLCQLXYBPW3LH7T' #input('Введите ключ доступа   -   ') 

In [23]:
# Считывание в массив наименований компаний с .txt
"""symbolCode = []
with open('CompaniesList.txt') as f:
    listTitleCompany = f.read()
    symbolCode = listTitleCompany.split(" ")"""
symbolCode = ['BA', 'AXP', 'CAT', 'GS', 'HD', 'F', 'DAL', 'BAC', 'WFC', 'CPE']    
print(symbolCode)

['BA', 'AXP', 'CAT', 'GS', 'HD', 'F', 'DAL', 'BAC', 'WFC', 'CPE', 'OXY']


In [24]:
# Скачивание и запись в Dictionary Базы Данных
countCompany = len(symbolCode)
stockQuotes = {}
lengthDateBase = []
# Задаем задержку т.к. сайт отдает данные по 5ти фирмам в минуту
sleepFor = 0 
ts = TimeSeries(key=key)
for i in range (countCompany):
    sleepFor += 1
    if sleepFor % 5 == 0:
        time.sleep(61)
    # Запрос данных API
    data, meta_data = ts.get_intraday(symbol=symbolCode[i],interval='60min', outputsize='full')

    # Запись нужных параметров в массив
    times = []
    prices = []
    for key, price in data.items():
#        timeD = datetime.strptime(key, "%Y-%m-%d %H:%M:%S")
        timeD = key
        times.append(timeD)
        prices.append(price['1. open'])
    lengthDateBase.append(len(times))
    timesPrices = [times, prices]
    # Создание словаря
    stockQuotes[symbolCode[i]] = timesPrices
#    print(stockQuotes)
# print(stockQuotes)
print(symbolCode)
print(max(lengthDateBase))
print(len(symbolCode))


{'BA': [['2020-08-28 20:00:00', '2020-08-28 19:00:00', '2020-08-28 18:00:00', '2020-08-28 17:00:00', '2020-08-28 16:00:00', '2020-08-28 15:00:00', '2020-08-28 14:00:00', '2020-08-28 13:00:00', '2020-08-28 12:00:00', '2020-08-28 11:00:00', '2020-08-28 10:00:00', '2020-08-28 09:00:00', '2020-08-28 08:00:00', '2020-08-28 07:00:00', '2020-08-28 06:00:00', '2020-08-28 05:00:00', '2020-08-27 20:00:00', '2020-08-27 19:00:00', '2020-08-27 18:00:00', '2020-08-27 17:00:00', '2020-08-27 16:00:00', '2020-08-27 15:00:00', '2020-08-27 14:00:00', '2020-08-27 13:00:00', '2020-08-27 12:00:00', '2020-08-27 11:00:00', '2020-08-27 10:00:00', '2020-08-27 09:00:00', '2020-08-27 08:00:00', '2020-08-27 07:00:00', '2020-08-27 06:00:00', '2020-08-27 05:00:00', '2020-08-26 20:00:00', '2020-08-26 19:00:00', '2020-08-26 18:00:00', '2020-08-26 17:00:00', '2020-08-26 16:00:00', '2020-08-26 15:00:00', '2020-08-26 14:00:00', '2020-08-26 13:00:00', '2020-08-26 12:00:00', '2020-08-26 11:00:00', '2020-08-26 10:00:00', '2

### **Преоброзование данных в удобную для анализа форму**

In [25]:
# Создание копии Данных
stockQuotesReserve = stockQuotes.copy() 

In [26]:
pdStockQuotes = []
for key, timePrice in stockQuotes.items():
    times = timePrice[0]
    prices = timePrice[1]
    for i in range (len(times)):
        
        pdStockQuotes.append([key, times[i], prices[i]])

In [27]:
# Запись промежуточных данных в файл
with open('TimeSeries.json', 'w') as f:
    f.write(json.dumps(pdStockQuotes))

In [28]:
AllDataBase = pd.read_json('/content/TimeSeries.json')
AllDataBase.columns = ['Company', 'DataTime', 'Price']
#AllDataBase = AllDataBase.sort_values(by=['DateTime'], ascending=False)
AllDataBase

,Company,DataTime,Price
0,BA,2020-08-28 20:00:00,175.70
1,BA,2020-08-28 19:00:00,175.86
2,BA,2020-08-28 18:00:00,175.66
3,BA,2020-08-28 17:00:00,175.80
4,BA,2020-08-28 16:00:00,175.66
...,...,...,...
6447,OXY,2020-07-06 09:00:00,18.25
6448,OXY,2020-07-06 08:00:00,18.40
6449,OXY,2020-07-06 07:00:00,18.35
6450,OXY,2020-07-06 06:00:00,18.20


In [78]:
allData = (AllDataBase
       .pivot(index="DataTime", columns="Company", values="Price")
       .rename_axis(None, axis=1)
       .reset_index())
allData

,DataTime,AXP,BA,BAC,CAT,CPE,DAL,F,GS,HD,OXY,WFC
0,2020-07-06 05:00:00,95.50,185.15,23.78,NaN,1.1900,28.3500,6.07,NaN,NaN,18.11,26.000
1,2020-07-06 06:00:00,NaN,184.70,23.90,NaN,1.2200,28.5000,6.11,NaN,NaN,18.20,26.050
2,2020-07-06 07:00:00,97.00,185.15,23.95,NaN,1.2000,28.6500,6.12,202.370,NaN,18.35,26.100
3,2020-07-06 08:00:00,96.14,185.67,23.92,129.6500,1.2100,28.5100,6.15,202.710,252.0000,18.40,26.100
4,2020-07-06 09:00:00,96.00,184.10,23.80,130.9109,1.2099,28.4767,6.14,202.735,252.2398,18.25,25.890
...,...,...,...,...,...,...,...,...,...,...,...,...
635,2020-08-28 16:00:00,102.60,175.66,26.28,144.3500,7.0000,31.9100,6.93,209.110,286.4900,13.18,24.615
636,2020-08-28 17:00:00,102.54,175.80,26.30,143.6300,6.9800,31.9900,6.94,207.640,286.2900,13.13,24.690
637,2020-08-28 18:00:00,102.54,175.66,26.26,143.7100,6.9600,31.9800,6.93,207.740,286.2900,13.10,24.660
638,2020-08-28 19:00:00,102.50,175.86,26.26,NaN,NaN,31.9300,6.93,207.840,286.4900,13.09,24.660


## **Обработка и анализ данных**

In [79]:
# Проверка на отсутствующие значения
allData_na = allData.isnull().sum()
allData_na = allData_na.drop(allData_na[allData_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :allData_na})
missing_data.iloc[0:10]

,Missing Ratio
CAT,144
AXP,130
HD,120
GS,106
CPE,40
F,36
WFC,9
DAL,2
BAC,1


In [85]:
noneDataLabel = missing_data.index.values
#noneDataCount = missing_data.reset_index(level = 0)['Missing Ratio'].values
noneDataLabel

array([], dtype=object)

In [82]:
# Заполнение отсутствующих значений
allData = allData.fillna(method='ffill')
allData = allData.fillna(method='bfill')
allData.iloc[0:5]

,DataTime,AXP,BA,BAC,CAT,CPE,DAL,F,GS,HD,OXY,WFC
0,2020-07-06 05:00:00,95.50,185.15,23.78,129.6500,1.1900,28.3500,6.07,202.370,252.0000,18.11,26.00
1,2020-07-06 06:00:00,95.50,184.70,23.90,129.6500,1.2200,28.5000,6.11,202.370,252.0000,18.20,26.05
2,2020-07-06 07:00:00,97.00,185.15,23.95,129.6500,1.2000,28.6500,6.12,202.370,252.0000,18.35,26.10
3,2020-07-06 08:00:00,96.14,185.67,23.92,129.6500,1.2100,28.5100,6.15,202.710,252.0000,18.40,26.10
4,2020-07-06 09:00:00,96.00,184.10,23.80,130.9109,1.2099,28.4767,6.14,202.735,252.2398,18.25,25.89


In [83]:
# Проверка на отсутствующие значения
allData_na = allData.isnull().sum()
allData_na = allData_na.drop(allData_na[allData_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :allData_na})
missing_data.iloc[0:10]

,Missing Ratio
